In [5]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import csv
import numpy as np
import cv2
from tqdm import tqdm

from edge_detection import edgeDetection
from interest_point_detection import display_interest_points_matrix
from detect_laser import *
from torchinfo import summary

In [9]:
# Prepare data 

data_path = Path("./data")
csv_list = list(data_path.glob("*.csv"))
file_map = dict()
i = 0
for csv_file in csv_list:   
    with open(csv_file, newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        header = next(reader)

        # First 3 of every file will be for testing
        next(reader)
        next(reader)
        next(reader)
        
        for row in reader:
        
            if len(row) > 0:
                file_map[i] = dict()
                file_map[i]["name"] = row[0]
                
                for j in range(1, 3):
                    file_map[i][header[j]] = row[j]

                i+=1

In [7]:
# Create the mask using both laser calibrations

calib1 = Path("old_configs/calibration-output.dat")
lcalib1 = Path("old_configs/laser-calibration-output-4-12-bot.dat")

calib2 = Path("old_configs/fsl-01d-lens.dat")
lcalib2 = Path("old_configs/laser-calibration-output-7-13.dat")

calib3 = Path("old_configs/fsl-01d-lens-raw.dat")
lcalib3 = Path("old_configs/laser-calibration-output-7-13.dat")

shape = (3000, 4000)
mask1 = get_mask(lcalib1, calib1, shape)
mask2 = get_mask(lcalib2, calib2, shape)
mask3 = get_mask(lcalib3, calib3, shape)

fat_mask = return_mask_stack([mask1, mask2, mask3])

# cv2.namedWindow("img", cv2.WINDOW_NORMAL)
# cv2.imshow("img", fat_mask*255)
# cv2.waitKey(0)
# cv2.destroyAllWindows()   

In [ ]:
for i in range(350):
    img = cv2.imread(file_map[i]["name"])
    img = img[:3000,:4000,:]
    img = get_masked_image_with_mask(img, fat_mask)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    # set lower and upper color limits
    lower_val = np.array([0,0,200])
    upper_val = np.array([255,50,255])
    # Threshold the HSV image 
    mask = cv2.inRange(hsv, lower_val, upper_val)
    # remove noise
    kernel =  np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    # find contours in mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # draw contours
    for cnt in contours:
        cv2.drawContours(img,[cnt],0,(0,0,255),thickness=-1)
    #show image

    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

In [ ]:
for i in range(350):
    img = cv2.imread(file_map[i]["name"])
    img = img[:3000,:4000,:]
    img = get_masked_image_with_mask(img, fat_mask)

    interest_points = display_interest_points_matrix(img)

    cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

In [8]:
# Check to make sure everything is working with the data loading
for i in file_map:
    
    print(f"{i}: {file_map[i]['name']} has laser at coordinates ({file_map[i]['laser.x']}, {file_map[i]['laser.y']})")

0: data/Green Laser Test-rectified/P7170011.JPG has laser at coordinates (2063, 1445)
1: data/Green Laser Test-rectified/P7170010.JPG has laser at coordinates (2061, 1436)
2: data/Green Laser Test-rectified/P7170002.JPG has laser at coordinates (2014, 1276)
3: data/Green Laser Test-rectified/P7170007.JPG has laser at coordinates (2058, 1415)
4: data/Green Laser Test-rectified/P7170003.JPG has laser at coordinates (2027, 1312)
5: data/Green Laser Test-rectified/P7170001.JPG has laser at coordinates (2008, 1268)
6: data/Green Laser Test-rectified/P7170005.JPG has laser at coordinates (2048, 1384)
7: data/Green Laser Test-rectified/P7170004.JPG has laser at coordinates (2037, 1348)
8: data/Green Laser Test-rectified/P7170006.JPG has laser at coordinates (2054, 1403)
9: data/fsl-01d-07-25-23/fred-rectified/P7130192.PNG has laser at coordinates (1936, 1354)
10: data/fsl-01d-07-25-23/fred-rectified/P7130166.PNG has laser at coordinates (1795, 902)
11: data/fsl-01d-07-25-23/fred-rectified/P71

In [4]:
class LaserDetectionDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, file_map, fat_mask):
        """
        Arguments:
            file_map (string): map of all laser positions
        """
        self.file_map = file_map
        self.fat_mask = fat_mask

    def __len__(self):

        return len(self.file_map)

    def __getitem__(self, idx):

        file_name = self.file_map[idx]["name"]
        laser_x = int(self.file_map[idx]["laser.x"])
        laser_y = int(self.file_map[idx]["laser.y"])

        laser_coords = np.array([laser_x, laser_y])


        img = cv2.imread(file_name)
        img = img[:3000,:4000,:]
        masked_image = get_masked_image_with_mask(img, self.fat_mask)

        hsv = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV) 
        # set lower and upper color limits
        lower_val = np.array([0,0,200])
        upper_val = np.array([255,50,255])
        # Threshold the HSV image 
        mask = cv2.inRange(hsv, lower_val, upper_val)
        # remove noise
        kernel =  np.ones((5,5),np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        # find contours in mask
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # draw contours
        for cnt in contours:
            cv2.drawContours(masked_image,[cnt],0,(0,0,255),thickness=-1)

        return masked_image[0:1700, 1200:2300,:], laser_coords

In [41]:
laser_dataset = LaserDetectionDataset(file_map = file_map, fat_mask=fat_mask)
laser_dataloader = DataLoader(laser_dataset)

for i, sample in tqdm(enumerate(laser_dataloader)):

    resized_img, laser_coords = sample
    print(resized_img.shape)
    print(laser_coords)
    laser_np = (laser_coords.numpy()[0][0]-1200, laser_coords.numpy()[0][1])

    display = np.reshape(resized_img.numpy(), (1700, 1100, 3))

    # print(laser_np)
    cv2.circle(display, center=laser_np, radius = 20, thickness=10, color=(0,0,255))
    
    cv2.namedWindow("burer", cv2.WINDOW_NORMAL)
    cv2.imshow("burer", display)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    break
    

0it [00:00, ?it/s]

torch.Size([1, 1700, 1100, 3])
tensor([[2060, 1424]])


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 34459, resource id: 0, major code: 140 (Unknown), minor code: 20
0it [00:05, ?it/s]


In [1]:
class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(59840000, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.linear1(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
model = Net((1700, 1100, 3))
batch_size = 1
summary(model, input_size=(batch_size, 3, 1700, 1100), device="cpu")